In [1]:
import os
import glob
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns




accuracy_dir = "/home/ainedineen/blurry_vision/pytorch_untrained_models/imagenet/exp1_blurry_vision/exp1_accuracy"

_train_test_files = glob.glob(os.path.join(accuracy_dir, "train-*_test-*.csv"))


acc_df = pd.DataFrame()
# Load all csvs in directpry /shape_bias_train-*_test-0*_16_class_IN.csv
for file in _train_test_files:
    print(file)
    df = pd.read_csv(file)
    print(df.head())
    # Get the name of the file without the extension
    file_name = os.path.splitext(os.path.basename(file))[0]
    print(f'colnames: {df.columns}')

    # drop column 'Unnamed: 0' 
    if 'Unnamed: 0' in df.columns:
        df = df.drop(columns=['Unnamed: 0'])

    # drop val_images_blurred and val_blur_sigma
    if 'val_images_blurred' in df.columns:
        df = df.drop(columns=['val_images_blurred'])
    if 'val_blur_sigma' in df.columns:
        df = df.drop(columns=['val_blur_sigma'])

    train_blur = float(file.split('/')[-1].split('train-')[1].split('_')[0])
    test_blur = float(file.split('/')[-1].split('test-')[1].split('.')[0])
    print(f"Train blur: {train_blur}, Test blur: {test_blur}")

    # add column 'train_blur' and 'test_blur'
    df['train_blur'] = train_blur
    df['test_blur'] = test_blur



    # append the dataframe to acc_df
    acc_df = pd.concat([acc_df, df], ignore_index=True)

print(acc_df.head())

    # ['Unnamed: 0', 'model_pth', 'epoch', 'top1acc', 'top5acc', 'loss',
    #    'val_images_blurred', 'val_blur_sigma']


/home/ainedineen/blurry_vision/pytorch_untrained_models/imagenet/exp1_blurry_vision/exp1_accuracy/train-1_test-1.csv
   Unnamed: 0                                          model_pth  epoch  \
0           0  /data/blurry_vision_sup_RN50/supervised_resnet...     60   

   top1acc  top5acc    loss  val_images_blurred  val_blur_sigma  
0   66.858    88.08  1.3303                   0               0  
colnames: Index(['Unnamed: 0', 'model_pth', 'epoch', 'top1acc', 'top5acc', 'loss',
       'val_images_blurred', 'val_blur_sigma'],
      dtype='object')
Train blur: 1.0, Test blur: 1.0
/home/ainedineen/blurry_vision/pytorch_untrained_models/imagenet/exp1_blurry_vision/exp1_accuracy/train-1.5_test-4.csv
   Unnamed: 0                                          model_pth  epoch  \
0           0  /data/blurry_vision_sup_RN50/supervised_resnet...     60   

   top1acc  top5acc    loss  val_images_blurred  val_blur_sigma  
0   28.782   51.432  3.6433                   0               0  
colnames: Ind

In [2]:
acc_df

,model_pth,epoch,top1acc,top5acc,loss,train_blur,test_blur
0,/data/blurry_vision_sup_RN50/supervised_resnet...,60,66.858,88.080,1.3303,1.0,1.0
1,/data/blurry_vision_sup_RN50/supervised_resnet...,60,28.782,51.432,3.6433,1.5,4.0
2,/data/blurry_vision_sup_RN50/supervised_resnet...,60,43.070,67.556,2.8408,1.0,0.0
3,/data/blurry_vision_sup_RN50/supervised_resnet...,60,60.202,83.580,1.6689,1.0,1.0
4,/data/blurry_vision_sup_RN50/supervised_resnet...,60,35.186,59.414,3.1630,2.0,4.0
...,...,...,...,...,...,...,...
59,/data/blurry_vision_sup_RN50/supervised_resnet...,60,6.088,14.894,8.0430,1.5,0.0
60,/data/blurry_vision_sup_RN50/supervised_resnet...,60,1.968,6.200,10.0080,2.0,0.0
61,/data/blurry_vision_sup_RN50/supervised_resnet...,60,58.988,82.054,1.7403,1.5,2.0
62,/data/blurry_vision_sup_RN50/supervised_resnet...,60,0.706,2.388,11.2170,3.0,1.0


In [11]:

def plot_acc_heatmap(acc_df, metric_to_plot, metric_label, plot_label, save_path, mask=False):

    df = acc_df.copy()

    # Pivot the data for heatmap
    heatmap_data = df.pivot_table(
        index='test_blur', 
        columns='train_blur', 

        values=metric_to_plot,
        aggfunc='mean',  # In case there are multiple entries for same (test, train)
    )
    # mask = np.tril(np.ones_like(heatmap_data, dtype=bool))

    # Sort axes if needed
    heatmap_data = heatmap_data.sort_index(ascending=False)  # flip y-axis
    heatmap_data = heatmap_data.sort_index(axis=1)           # sort x-axis


    # Create the heatmap
    plt.figure(figsize=(10, 7))

    if mask:

        # Mask Here - mask lower triangle (diagonal runs from bottom left to top right)
        mask = np.tril(np.ones_like(heatmap_data, dtype=bool), k=-1)
        # fli[p the mask to show upper triangle
        mask = np.flip(mask, axis=1)

        
        sns.heatmap(heatmap_data, annot=True, cmap='viridis', fmt=".2f",
                    cbar_kws={'label': metric_label},
                    mask=mask
                    )
    else:
        sns.heatmap(heatmap_data, annot=True, cmap='viridis', fmt=".2f",
                    cbar_kws={'label': metric_label}
                    )
                
    # plt.title('Shape Bias Heatmap')
    # plt.title('Proportion of Correct Labels for Cue Conflict Stimuli')
    plt.title(plot_label)
    plt.xlabel('Train Blur')
    plt.ylabel('Test Blur')
    plt.tight_layout()
    # plt.show()
    # plt.savefig('acc_by_train_and_test_blur.png', dpi=300)
    plt.savefig(save_path, dpi=300)

In [ ]:
plot_acc_heatmap(acc_df, 'top1acc', 'Top-1 Accuracy', 'Top-1 Accuracy by Train and Test Blur', 'top1acc_by_train_and_test_blur.png')
plot_acc_heatmap(acc_df, 'top5acc', 'Top-5 Accuracy', 'Top-5 Accuracy by Train and Test Blur', 'top5acc_by_train_and_test_blur.png')
# plot_acc_heatmap(acc_df, 'loss', 'Loss', 'Loss by Train and Test Blur', 'loss_by_train_and_test_blur.png', mask=True)
